In [1]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [2]:
locator = Nominatim(user_agent='RooseveltIsland')
location = locator.geocode("Cornell Tech")

In [3]:
print(location.address)
#print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
lat = location.latitude
long = location.longitude
alt = location.altitude
print("Latitude: ", lat)
print("Longitude: ", long)
print("Altitude: ", alt)

Cornell Tech, Road 3, Manhattan Community Board 8, Manhattan, New York County, New York, 11109, United States of America
Latitude:  40.75483025
Longitude:  -73.9563665209901
Altitude:  0.0


In [4]:
df = pd.read_excel("addresses.xlsx")
df.head()

,Business,Website,Farmers or Buyers,"Farmer(1), Buyer(0)",Street,City,State,Zip,latitude,longitude
0,Stony Point Center,https://stonypointcenter.org/,Buyer,0,17 Cricketown Road,Stony Point,NY,10980,NaN,NaN
1,Vassar College,https://www.vassar.edu/,Buyer,0,124 Raymond Avenue,Poughkeepsie,NY,12604,NaN,NaN
2,Bard College,http://www.bard.edu/,Buyer,0,1400 Annandale Road,Annandale-on-the-Hudson,NY,12504,NaN,NaN
3,SUNY New Paltz,https://www.newpaltz.edu/,Buyer,0,1 Hawk Drive,New Paltz,NY,12561,NaN,NaN
4,Woodland Pond,https://wpatnp.org/,Buyer,0,100 Woodland Pond Circle,New Paltz,NY,12561,NaN,NaN


In [5]:
df['address'] = df['Street'].astype(str) + ', ' + \
                df['City '].astype(str) + ', ' + \
                df['State'].astype(str) + ', ' + \
                df['Zip'].astype(str) + ', ' + 'United States of America'

df.head()

,Business,Website,Farmers or Buyers,"Farmer(1), Buyer(0)",Street,City,State,Zip,latitude,longitude,address
0,Stony Point Center,https://stonypointcenter.org/,Buyer,0,17 Cricketown Road,Stony Point,NY,10980,NaN,NaN,"17 Cricketown Road, Stony Point, NY, 10980, Un..."
1,Vassar College,https://www.vassar.edu/,Buyer,0,124 Raymond Avenue,Poughkeepsie,NY,12604,NaN,NaN,"124 Raymond Avenue, Poughkeepsie, NY, 12604, U..."
2,Bard College,http://www.bard.edu/,Buyer,0,1400 Annandale Road,Annandale-on-the-Hudson,NY,12504,NaN,NaN,"1400 Annandale Road, Annandale-on-the-Hudson, ..."
3,SUNY New Paltz,https://www.newpaltz.edu/,Buyer,0,1 Hawk Drive,New Paltz,NY,12561,NaN,NaN,"1 Hawk Drive, New Paltz, NY, 12561, United Sta..."
4,Woodland Pond,https://wpatnp.org/,Buyer,0,100 Woodland Pond Circle,New Paltz,NY,12561,NaN,NaN,"100 Woodland Pond Circle, New Paltz, NY, 12561..."


In [6]:
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['address'].apply(geocode)

In [7]:
df['latitude'] = df['location'].apply(lambda loc: (loc.latitude) if loc else None)
df['longitude'] = df['location'].apply(lambda loc: (loc.longitude) if loc else None)
df.head()

,Business,Website,Farmers or Buyers,"Farmer(1), Buyer(0)",Street,City,State,Zip,latitude,longitude,address,location
0,Stony Point Center,https://stonypointcenter.org/,Buyer,0,17 Cricketown Road,Stony Point,NY,10980,41.232752,-73.994029,"17 Cricketown Road, Stony Point, NY, 10980, Un...","(17, Cricketown Road, Stony Point, Town of Sto..."
1,Vassar College,https://www.vassar.edu/,Buyer,0,124 Raymond Avenue,Poughkeepsie,NY,12604,41.686786,-73.895697,"124 Raymond Avenue, Poughkeepsie, NY, 12604, U...","(Vassar College, 124, Raymond Avenue, Arlingto..."
2,Bard College,http://www.bard.edu/,Buyer,0,1400 Annandale Road,Annandale-on-the-Hudson,NY,12504,42.021950,-73.907480,"1400 Annandale Road, Annandale-on-the-Hudson, ...","(Kline Commons, 1400, Annandale Road, Annandal..."
3,SUNY New Paltz,https://www.newpaltz.edu/,Buyer,0,1 Hawk Drive,New Paltz,NY,12561,41.740459,-74.080925,"1 Hawk Drive, New Paltz, NY, 12561, United Sta...","(Hawk Drive, New Paltz, Town of New Paltz, Uls..."
4,Woodland Pond,https://wpatnp.org/,Buyer,0,100 Woodland Pond Circle,New Paltz,NY,12561,41.757368,-74.063018,"100 Woodland Pond Circle, New Paltz, NY, 12561...","(Woodland Pond at New Paltz, 100, Woodland Pon..."


In [8]:
df1 = df[pd.notnull(df['latitude'])]
df1

,Business,Website,Farmers or Buyers,"Farmer(1), Buyer(0)",Street,City,State,Zip,latitude,longitude,address,location
0,Stony Point Center,https://stonypointcenter.org/,Buyer,0,17 Cricketown Road,Stony Point,NY,10980,41.232752,-73.994029,"17 Cricketown Road, Stony Point, NY, 10980, Un...","(17, Cricketown Road, Stony Point, Town of Sto..."
1,Vassar College,https://www.vassar.edu/,Buyer,0,124 Raymond Avenue,Poughkeepsie,NY,12604,41.686786,-73.895697,"124 Raymond Avenue, Poughkeepsie, NY, 12604, U...","(Vassar College, 124, Raymond Avenue, Arlingto..."
2,Bard College,http://www.bard.edu/,Buyer,0,1400 Annandale Road,Annandale-on-the-Hudson,NY,12504,42.021950,-73.907480,"1400 Annandale Road, Annandale-on-the-Hudson, ...","(Kline Commons, 1400, Annandale Road, Annandal..."
3,SUNY New Paltz,https://www.newpaltz.edu/,Buyer,0,1 Hawk Drive,New Paltz,NY,12561,41.740459,-74.080925,"1 Hawk Drive, New Paltz, NY, 12561, United Sta...","(Hawk Drive, New Paltz, Town of New Paltz, Uls..."
4,Woodland Pond,https://wpatnp.org/,Buyer,0,100 Woodland Pond Circle,New Paltz,NY,12561,41.757368,-74.063018,"100 Woodland Pond Circle, New Paltz, NY, 12561...","(Woodland Pond at New Paltz, 100, Woodland Pon..."
6,Taliaferro Farms,https://www.taliaferrofarms.com/,Farmer,1,187 Plains Road,New Paltz,NY,12561,41.744003,-74.091190,"187 Plains Road, New Paltz, NY, 12561, United ...","(Plains Road, New Paltz, Town of New Paltz, Ul..."
8,Poughkeepsie Farm Project,https://www.farmproject.org/,Farmer,1,51 Vassar Farm Lane,Poughkeepsie,NY,12603,41.680507,-73.896173,"51 Vassar Farm Lane, Poughkeepsie, NY, 12603, ...","(Vassar Farm Lane, Arlington, Town of Poughkee..."
10,Evolutionary Organics,https://www.evolutionaryorganics.com/,Farmer,1,283 Springtown Road,New Paltz,NY,12561,41.760486,-74.097562,"283 Springtown Road, New Paltz, NY, 12561, Uni...","(Springtown Road, New Paltz, Town of New Paltz..."
11,Huguenot Street Farm,https://www.huguenotfarm.com/,Farmer,1,205 Huguenot Street,New Paltz,NY,12561,41.763388,-74.083527,"205 Huguenot Street, New Paltz, NY, 12561, Uni...","(205, Huguenot Street, New Paltz, Town of New ..."
12,Ironwood Farm,https://www.ironwood.farm/,Farmer,1,103 County Route 9,Ghent,NY,12075,42.331852,-73.611646,"103 County Route 9, Ghent, NY, 12075, United S...","(103, County Road 9, Ghent, Town of Ghent, Col..."


In [9]:
map1 = folium.Map(location=[0.0,0.0], tiles='cartodbpositron',zoom_start=5)
df1.apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']]).add_to(map1), axis=1)
map1

In [10]:
folium_map = folium.Map(location=[0.0,0.0],
                        zoom_start=5,
                        tiles='CartoDB dark_matter')


FastMarkerCluster(data=list(zip(df1['latitude'].values, df1['longitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

In [11]:
df2 = df1[['Business','latitude','longitude','location']]
df2

,Business,latitude,longitude,location
0,Stony Point Center,41.232752,-73.994029,"(17, Cricketown Road, Stony Point, Town of Sto..."
1,Vassar College,41.686786,-73.895697,"(Vassar College, 124, Raymond Avenue, Arlingto..."
2,Bard College,42.021950,-73.907480,"(Kline Commons, 1400, Annandale Road, Annandal..."
3,SUNY New Paltz,41.740459,-74.080925,"(Hawk Drive, New Paltz, Town of New Paltz, Uls..."
4,Woodland Pond,41.757368,-74.063018,"(Woodland Pond at New Paltz, 100, Woodland Pon..."
6,Taliaferro Farms,41.744003,-74.091190,"(Plains Road, New Paltz, Town of New Paltz, Ul..."
8,Poughkeepsie Farm Project,41.680507,-73.896173,"(Vassar Farm Lane, Arlington, Town of Poughkee..."
10,Evolutionary Organics,41.760486,-74.097562,"(Springtown Road, New Paltz, Town of New Paltz..."
11,Huguenot Street Farm,41.763388,-74.083527,"(205, Huguenot Street, New Paltz, Town of New ..."
12,Ironwood Farm,42.331852,-73.611646,"(103, County Road 9, Ghent, Town of Ghent, Col..."


In [12]:
from geopy.distance import geodesic
Cornell_Tech = (lat,long,alt)
df2['point'] = df2['location'].apply(lambda loc: tuple(loc.point) if loc else None)

/Users/royp94/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
distance = {}
for i in range (12):
    distance.update({df2.iloc[i,0]: geodesic(Cornell_Tech,df2.iloc[i,4]).miles})

In [21]:
print(sorted(distance.items(), key=lambda x:x[1]))

[('Cropsey Community Farm', 26.271809764192692), ('Stony Point Center', 33.038028260939036), ('Second Wind CSA', 63.64570343662168), ('Poughkeepsie Farm Project', 63.95634401768193), ('Vassar College', 64.39036968874989), ('SUNY New Paltz', 68.32555749951923), ('Taliaferro Farms', 68.62169986788476), ('Woodland Pond', 69.40648705184582), ('Evolutionary Organics', 69.78748370163814), ('Huguenot Street Farm', 69.91363858864848), ('Bard College', 87.48130931779549), ('Ironwood Farm', 110.29129460484117)]
